### Import CSVs and convert to DataFrames

In [ ]:
# install packages
%%capture
packages = ["pandas", "kagglehub", "anthropic", "langchain", "langchain_community", "langchain_anthropic"]

for package in packages:
    try:
        __import__(package)
    except ImportError:
        import sys
        !{sys.executable} -m pip install {package}

# import data download and cleaning packages
import pandas as pd
import kagglehub
import warnings

# ignore all warnings
warnings.filterwarnings("ignore")

In [ ]:
# download and store path to datasets
path_cbb = kagglehub.dataset_download("andrewsundberg/college-basketball-dataset")
path_oscars = kagglehub.dataset_download("unanimad/the-oscar-award")

In [ ]:
# read cbb.CSV
cbb = pd.read_csv(path_cbb + '/cbb.csv')

# make all columns lowercase
cbb.columns = cbb.columns.str.lower()

# fix data for teams that didn't make ncaa tournament
cbb['seed'] = cbb['seed'].apply(
    lambda x: str(int(x)) if pd.notna(x) else "no_postseason")

# make all object data lowercase
for col in cbb.select_dtypes(include=['object']).columns:
  cbb[col] = cbb[col].str.lower()

# map 'postseason' outcomes to descriptive categories
outcome_map = {
    'r68': 'round_68',
    'r64': 'round_64',
    'r32': 'round_32',
    's16': 'sweet_16',
    'e8': 'elite_8',
    'f4': 'final_4',
    '2nd': 'runner_up',
    'champions': 'champions'
}

cbb['postseason'] = cbb['postseason'].apply(lambda x: outcome_map.get(x, x))


# show preview
cbb.head()

,team,conf,g,w,adjoe,adjde,barthag,efg_o,efg_d,tor,...,ftrd,2p_o,2p_d,3p_o,3p_d,adj_t,wab,postseason,seed,year
0,north carolina,acc,40,33,123.3,94.9,0.9531,52.6,48.1,15.4,...,30.4,53.9,44.6,32.7,36.2,71.7,8.6,runner_up,1,2016
1,wisconsin,b10,40,36,129.1,93.6,0.9758,54.8,47.7,12.4,...,22.4,54.8,44.7,36.5,37.5,59.3,11.3,runner_up,1,2015
2,michigan,b10,40,33,114.4,90.4,0.9375,53.9,47.7,14.0,...,30.0,54.7,46.8,35.2,33.2,65.9,6.9,runner_up,3,2018
3,texas tech,b12,38,31,115.2,85.2,0.9696,53.5,43.0,17.7,...,36.6,52.8,41.9,36.5,29.7,67.5,7.0,runner_up,3,2019
4,gonzaga,wcc,39,37,117.8,86.3,0.9728,56.6,41.1,16.2,...,26.9,56.3,40.0,38.2,29.0,71.5,7.7,runner_up,1,2017


In [ ]:
# read oscars.CSV
oscars = pd.read_csv(path_oscars + '/the_oscar_award.csv')

# make all object data lowercase
for col in oscars.select_dtypes(include=['object']).columns:
  oscars[col] = oscars[col].str.lower()

# also convert boolean to lowercase
oscars['winner'] = oscars['winner'].map({True: 'true', False: 'false'})

# show preview
oscars.head(5)

,year_film,year_ceremony,ceremony,category,canon_category,name,film,winner
0,1927,1928,1,actor,actor in a leading role,richard barthelmess,the noose,false
1,1927,1928,1,actor,actor in a leading role,richard barthelmess,the patent leather kid,false
2,1927,1928,1,actor,actor in a leading role,emil jannings,the last command,true
3,1927,1928,1,actor,actor in a leading role,emil jannings,the way of all flesh,true
4,1927,1928,1,actress,actress in a leading role,louise dresser,a ship comes in,false


### SQLite Database

In [ ]:
# import sqlite3
import sqlite3

# create a connection to a new sqlite database file
conn = sqlite3.connect("cbb_oscars.db")

In [ ]:
# convert dataframes to sql tables
cbb.to_sql("cbb", conn, if_exists="replace", index=False)
oscars.to_sql("oscars", conn, if_exists="replace", index=False)

# close the connection
conn.close()

In [ ]:
# query, query-to-df functions

def query_db(query):

    conn = sqlite3.connect("cbb_oscars.db")
    cursor = conn.cursor()
    cursor.execute(query)
    results = cursor.fetchall()
    column_names = [description[0] for description in cursor.description]
    conn.close()

    return results, column_names

def q_to_df(query):
    conn = sqlite3.connect("cbb_oscars.db")
    df = pd.read_sql_query(query, conn)
    conn.close()

    return df

In [ ]:
# example
query = "SELECT * FROM cbb WHERE team = 'duke'"
results, columns = query_db(query)

# print results in a nice format
print("columns:", columns)
print("\nresults:")
for row in results:
    print(row)

columns: ['team', 'conf', 'g', 'w', 'adjoe', 'adjde', 'barthag', 'efg_o', 'efg_d', 'tor', 'tord', 'orb', 'drb', 'ftr', 'ftrd', '2p_o', '2p_d', '3p_o', '3p_d', 'adj_t', 'wab', 'postseason', 'seed', 'year']

results:
('duke', 'acc', 39, 35, 125.2, 90.6, 0.9764, 56.6, 46.5, 16.3, 18.6, 35.8, 30.2, 39.8, 23.9, 55.9, 46.3, 38.7, 31.4, 66.4, 10.7, 'champions', '1', 2015)
('duke', 'acc', 37, 29, 122.3, 92.9, 0.9592, 55.9, 46.4, 17.5, 17.3, 38.8, 30.0, 34.6, 22.3, 56.0, 45.4, 37.2, 32.0, 70.9, 6.6, 'elite_8', '2', 2018)
('duke', 'acc', 38, 32, 118.9, 89.2, 0.9646, 53.6, 45.0, 17.5, 19.4, 35.6, 29.5, 33.2, 24.0, 58.0, 45.0, 30.8, 29.9, 73.6, 11.2, 'elite_8', '1', 2019)
('duke', 'acc', 36, 30, 118.4, 91.5, 0.9507, 53.9, 45.5, 15.6, 20.4, 28.8, 32.3, 39.2, 32.7, 50.8, 46.2, 39.9, 29.0, 67.8, 7.5, 'elite_8', '2', 2013)
('duke', 'acc', 37, 28, 122.1, 95.6, 0.9436, 54.8, 47.4, 16.3, 17.3, 31.7, 29.8, 39.3, 31.2, 53.5, 48.9, 37.9, 29.3, 69.7, 8.6, 'round_32', '2', 2017)
('duke', 'acc', 35, 26, 125.9,

In [ ]:
# query to a df
q_to_df(query)

,team,conf,g,w,adjoe,adjde,barthag,efg_o,efg_d,tor,...,ftrd,2p_o,2p_d,3p_o,3p_d,adj_t,wab,postseason,seed,year
0,duke,acc,39,35,125.2,90.6,0.9764,56.6,46.5,16.3,...,23.9,55.9,46.3,38.7,31.4,66.4,10.7,champions,1,2015
1,duke,acc,37,29,122.3,92.9,0.9592,55.9,46.4,17.5,...,22.3,56.0,45.4,37.2,32.0,70.9,6.6,elite_8,2,2018
2,duke,acc,38,32,118.9,89.2,0.9646,53.6,45.0,17.5,...,24.0,58.0,45.0,30.8,29.9,73.6,11.2,elite_8,1,2019
3,duke,acc,36,30,118.4,91.5,0.9507,53.9,45.5,15.6,...,32.7,50.8,46.2,39.9,29.0,67.8,7.5,elite_8,2,2013
4,duke,acc,37,28,122.1,95.6,0.9436,54.8,47.4,16.3,...,31.2,53.5,48.9,37.9,29.3,69.7,8.6,round_32,2,2017
5,duke,acc,35,26,125.9,98.6,0.9432,53.8,49.3,14.6,...,40.8,50.3,50.3,39.5,30.7,66.7,6.5,round_64,3,2014
6,duke,acc,36,25,121.2,98.9,0.9122,53.7,49.5,14.3,...,25.3,51.0,48.9,38.5,34.0,68.7,4.2,sweet_16,4,2016
7,duke,acc,24,13,115.1,97.1,0.8758,53.7,51.3,18.3,...,31.9,54.3,50.4,35.1,35.5,68.5,-1.8,None,no_postseason,2021
8,duke,acc,39,32,122.7,96.0,0.9440,55.6,47.1,15.0,...,19.1,56.0,46.6,36.6,32.0,67.4,7.2,final_4,2,2022
9,duke,acc,36,27,113.3,92.9,0.9072,51.1,46.0,18.3,...,23.7,51.5,46.2,33.5,30.5,64.9,4.5,round_32,5,2023


In [ ]:
# example
query = "SELECT * FROM oscars WHERE film = 'oppenheimer'"
results, columns = query_db(query)

# print results in a nice format
print("columns:", columns)
print("\nresults:")
for row in results:
    print(row)

columns: ['year_film', 'year_ceremony', 'ceremony', 'category', 'canon_category', 'name', 'film', 'winner']

results:
(2023, 2024, 96, 'actor in a leading role', 'actor in a leading role', 'cillian murphy', 'oppenheimer', 'true')
(2023, 2024, 96, 'actor in a supporting role', 'actor in a supporting role', 'robert downey jr.', 'oppenheimer', 'true')
(2023, 2024, 96, 'actress in a supporting role', 'actress in a supporting role', 'emily blunt', 'oppenheimer', 'false')
(2023, 2024, 96, 'cinematography', 'cinematography', 'hoyte van hoytema', 'oppenheimer', 'true')
(2023, 2024, 96, 'costume design', 'costume design', 'ellen mirojnick', 'oppenheimer', 'false')
(2023, 2024, 96, 'directing', 'directing', 'christopher nolan', 'oppenheimer', 'true')
(2023, 2024, 96, 'film editing', 'film editing', 'jennifer lame', 'oppenheimer', 'true')
(2023, 2024, 96, 'makeup and hairstyling', 'makeup and hairstyling', 'luisa abel', 'oppenheimer', 'false')
(2023, 2024, 96, 'music (original score)', 'music (or

In [ ]:
# query to a df
q_to_df(query)

,year_film,year_ceremony,ceremony,category,canon_category,name,film,winner
0,2023,2024,96,actor in a leading role,actor in a leading role,cillian murphy,oppenheimer,true
1,2023,2024,96,actor in a supporting role,actor in a supporting role,robert downey jr.,oppenheimer,true
2,2023,2024,96,actress in a supporting role,actress in a supporting role,emily blunt,oppenheimer,false
3,2023,2024,96,cinematography,cinematography,hoyte van hoytema,oppenheimer,true
4,2023,2024,96,costume design,costume design,ellen mirojnick,oppenheimer,false
5,2023,2024,96,directing,directing,christopher nolan,oppenheimer,true
6,2023,2024,96,film editing,film editing,jennifer lame,oppenheimer,true
7,2023,2024,96,makeup and hairstyling,makeup and hairstyling,luisa abel,oppenheimer,false
8,2023,2024,96,music (original score),music (original score),ludwig göransson,oppenheimer,true
9,2023,2024,96,best picture,best picture,emma thomas/charles roven/christopher nolan,oppenheimer,true


### Anthropic

In [ ]:
# # install anthropic
# %%capture
# !pip install anthropic

In [ ]:
# # for secrets
# from google.colab import userdata

# # retrieve the API key
# api_key = userdata.get('ANTHROPIC_API_KEY')

# # use the key with the client
# import anthropic
# client = anthropic.Anthropic(api_key=api_key)

### Langchain

In [ ]:
# import anthropic, langchain, and colab packages
import anthropic
from langchain_anthropic import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableSequence
from google.colab import userdata

# setup api key retrieval
api_key = userdata.get('ANTHROPIC_API_KEY')

In [ ]:
# user_question = input("Ask a question about the database: ")
user_question = "what schools have won more than one national championship?"

In [ ]:
# LLM parameters
llm = ChatAnthropic(
    api_key=api_key,
    model_name="claude-3-haiku-20240307",
    max_tokens=400,
    temperature=0.1
)

In [ ]:
# create prompt template that uses table schemas / general instructions and adds additional instructions
prompt_template = ChatPromptTemplate.from_template(

"""

You are an AI assistant that converts natural language questions into SQL queries using the following database schema:

Table: cbb (College Basketball)
- team (TEXT): Division I college basketball team name (question might refer to this as 'school' or 'university')
- conf (TEXT): Athletic conference
- g (INTEGER): Games played
- w (INTEGER): Wins
- adjoe (REAL): Adjusted offensive efficiency
- adjde (REAL): Adjusted defensive efficiency
- barthag (REAL): Power ranking
- efg_o (REAL): Effective field goal percentage (offense)
- efg_d (REAL): Effective field goal percentage (defense)
- tor (REAL): Turnover rate (offense)
- tord (REAL): Turnover rate (defense)
- orb (REAL): Offensive rebound percentage
- drb (REAL): Defensive rebound percentage
- ftr (REAL): Free throw rate (offense)
- ftrd (REAL): Free throw rate (defense)
- 2p_o (REAL): 2-point percentage (offense)
- 2p_d (REAL): 2-point percentage (defense)
- 3p_o (REAL): 3-point percentage (offense)
- 3p_d (REAL): 3-point percentage (defense)
- adj_t (REAL): Adjusted tempo
- wab (REAL): Wins above bubble
- postseason (TEXT): Tournament outcome ('round_68', 'round_64', 'round_32', 'sweet_16', 'elite_8', 'final_4', 'runner_up', 'champions')
- seed (TEXT): NCAA seed as integer, 'no_postseason' if missing
- year (TEXT): Season year as integer, 'no_postseason' if missing

Table: oscars
- year_film (INTEGER): Year the film was released
- year_ceremony (INTEGER): Year of the award ceremony
- ceremony (INTEGER): Ceremony number
- category (TEXT): Award category group
- canon_category (TEXT): Award name
- name (TEXT): Name of the nominee
- film (TEXT): Film name
- winner (INTEGER): 1 if winner, 0 otherwise


Please follow these instructions/guidelines:

- Generate valid SQLite SQL queries ONLY using these table schemas, general instructions, and additional instructions to answer this question: {question}
- Return ONLY the SQL query without any explanations, comments, or markdown formatting.
- No template markers like {{sql_query}}


Additional Instructions:

- Always write valid SQL queries based on this schema.
- In SQLite, column names that start with numbers (like 2P_O or 3P_O) must be escaped with double quotes. For example: SELECT TEAM, "3P_O" FROM cbb
- Use LIMIT for wide queries.
- Don’t make up column names.
- All column names and values are lowercase. So, make sure query filters are lowercase even if original question contains capital letters

"""
)

Additional Instruction Notes:
* added line specifying that columns starting with numbers need to be wrapped in quotations
* added a line specifying postseason outcomes
* added a line clarifying that team, school, and university would all refer to the same columns
* added a line noting all columns and cell values are lowercase

In [ ]:
# create the SQL generation chain
sql_chain = prompt_template | llm


In [ ]:
# generate SQL query
sql_response = sql_chain.invoke({"question": user_question})
generated_sql = sql_response.content.strip()

print("Generated SQL query:")
print(generated_sql)

# Execute the SQL query using your existing function
print("\nExecuting query...")
results, columns = query_db(generated_sql)

print("\nColumns:", columns)
print("\nResults:")
for row in results:
    print(row)

Generated SQL query:
SELECT team, count(*) as num_championships
FROM cbb
WHERE postseason = 'champions'
GROUP BY team
HAVING count(*) > 1;

Executing query...

Columns: ['team', 'num_championships']

Results:
('connecticut', 3)
('villanova', 2)


In [ ]:
# initialize anthropic client
client = anthropic.Anthropic(api_key=api_key)

# build Claude explanation function
def get_claude_explanation(user_question, generated_sql, results, columns):
    """
    Send SQL results to Claude and get natural language explanation
    """
    # format results as readable string
    results_string = "Query Results:\n"
    results_string += ", ".join(columns) + "\n"
    for row in results:
        results_string += str(row) + "\n"

    # construct prompt
    prompt = f"""
    I asked the following question about a database: "{user_question}"

    This SQL query was executed to answer it:
    {generated_sql}

    Here are the results:
    {results_string}

    Please explain these results in simple natural language that directly answers my original question.
    Keep this fairly concise, and don't describe what the SQL query was. The SQL output is the important information needed to answer the original question.
    Also, don't reference the database or use the phrase 'the results show'.
    """

    # call Claude via Anthropic API
    response = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=200,
        temperature=0.3,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    # return Claude's explanation
    return response.content[0].text

In [ ]:
explanation = get_claude_explanation(user_question, generated_sql, results, columns)
print(explanation)

The schools that have won more than one national championship are Connecticut, with 3 championships, and Villanova, with 2 championships.
